# Query Shared MotherDuck Database

## Overview

This notebook demonstrates how to take a shared MotherDuck database and query it using python.

In [1]:
import os
import tomllib
import duckdb

# Load the TOML file
with open("config.toml", "rb") as file:
    config = tomllib.load(file)

# Extract the MD_SERVICE_TOKEN from the loaded config
md_service_token = config['mother-duck']['MD_SERVICE_TOKEN']

os.environ['motherduck_token'] = md_service_token

con = duckdb.connect(f'md:?motherduck_token={md_service_token}') 

In [2]:
md_shared_db = config['mother-duck']['MD_SHARED_DB']

try:
    con.sql(f"ATTACH '{md_shared_db}' AS md_shared_db")
except Exception as e:
    if "already attached" in str(e):
        print("Already attached")
    else:
        # Handle other exceptions if needed
        print("An error occurred:", e)

Already attached


In [3]:
import polars as pl
import pandas as pd
import ibis

#con.sql("USE voc_omop54_shared")

# Polars
#df = con.sql("SELECT * FROM concept").pl()

# Pandas
#df = con.sql("SELECT * FROM concept").df()

# Ibis
duckdb_con = ibis.connect(f'duckdb://md:voc_omop54_shared')

In [4]:
ibis.options.interactive = True
duckdb_con.list_tables()

['concept',
 'concept_ancestor',
 'concept_class',
 'concept_recommended',
 'concept_relationship',
 'concept_synonym',
 'vocabulary']

In [5]:
table_of_interest = 'concept'

df_concept = duckdb_con.table(f'{table_of_interest}')
row_count = df_concept.count()

print(f'{table_of_interest} has a total of {row_count} rows.')

concept has a total of 9441375 rows.


In [6]:
df_concept.head(5)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ concept_id ┃ concept_name                                                           ┃ domain_id ┃ vocabulary_id ┃ concept_class_id ┃ standard_concept ┃ concept_code ┃ valid_start_date ┃ valid_end_date ┃ invalid_reason ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ int32      │ string                                                                 │ string    │ string        │ string           │ string           │ string       │ date             │ date           │ string         │
├────────────┼────────────────────────────────────────────────────────────────────────┼───────────┼───────────────┼──────────────────┼──────────────────┼──────────────┼──────────────────┼────────────────┼────────────────┤
│   45053734 │ Etodolac 300 MG Oral Capsule [Lodine]                                  │ Drug      │ NDC           │ 11-digit NDC     │ NULL             │ 54569326401  │ 2007-06-01       │ 2011-01-01     │ D              │
│   45053735 │ Cephalexin 500 MG Oral Capsule                                         │ Drug      │ NDC           │ 11-digit NDC     │ NULL             │ 54569332406  │ 2007-06-01       │ 2012-12-01     │ D              │
│   45053736 │ Amoxicillin 500 MG Oral Capsule                                        │ Drug      │ NDC           │ 11-digit NDC     │ NULL             │ 54569333501  │ 2007-06-01       │ 2012-12-01     │ D              │
│   45053737 │ 12 HR Hyoscyamine Sulfate 0.375 MG Extended Release Capsule [Levsinex] │ Drug      │ NDC           │ 11-digit NDC     │ NULL             │ 54569349600  │ 2011-01-01       │ 2013-07-01     │ D              │
│   45053738 │ Chloroprocaine hydrochloride 10 MG/ML Injectable Solution [Nesacaine]  │ Drug      │ NDC           │ 11-digit NDC     │ NULL             │ 54569350000  │ 2010-06-01       │ 2013-07-01     │ D              │
└────────────┴────────────────────────────────────────────────────────────────────────┴───────────┴───────────────┴──────────────────┴──────────────────┴──────────────┴──────────────────┴────────────────┴────────────────┘